<a href="https://colab.research.google.com/github/bhavu93/CE888_2021/blob/main/Lab_4/my_reccomender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv',header=None)
print(df)

       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 101 columns]


In [3]:
df=df.drop(df.columns[0],1)
print(df)

         1      2      3      4     5    ...    96     97     98     99     100
0      -7.82   8.79  -9.66  -8.16 -7.52  ...  99.00  -5.63  99.00  99.00  99.00
1       4.08  -0.29   6.36   4.37 -2.38  ...  -2.14   3.06   0.34  -4.32   1.07
2      99.00  99.00  99.00  99.00  9.03  ...  99.00  99.00  99.00  99.00  99.00
3      99.00   8.35  99.00  99.00  1.80  ...  99.00  99.00  99.00  99.00  99.00
4       8.50   4.61  -4.17  -5.39  1.36  ...   1.55   3.11   6.55   1.80   1.60
...      ...    ...    ...    ...   ...  ...    ...    ...    ...    ...    ...
24978   0.44   7.43   9.08   2.33  3.20  ...   9.03   6.55   8.69   8.79   7.43
24979   9.13  -8.16   8.59   9.08  0.87  ...  -8.20  -7.23  -8.59   9.13   8.45
24980  99.00  99.00  99.00  99.00 -7.77  ...  99.00  99.00  99.00  99.00  99.00
24981  99.00  99.00  99.00  99.00 -9.71  ...  99.00  99.00  99.00  99.00  99.00
24982   2.43   2.67  -3.98   4.27 -2.28  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 100 columns]


In [4]:
arr = df.values
print(arr.shape)

(24983, 100)


In [5]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1810455 (1810455,)


In [6]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [7]:
new_arr, idx = replace(arr, 0.1)

In [8]:
arr[idx[0][0], idx[1][0]]

7.77

In [9]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [10]:
n_latent_factors = 15

user_ratings = df.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [11]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = new_arr[user_id][item_id]
                if (rating!=99):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if (iteration % 10) == 0:
        print(mse)

In [ ]:
sgd()

25.868490721436736
24.93541357680772
24.883973437071976
24.837181974127198
24.79271327864329
24.748551489582617
24.70265901021685
24.65262943740669
24.59528059641089
24.526140316351267
24.438783356201284
24.32401808694459
24.16903141463786
23.956847019261627
23.66689872139798
23.278086784073118
22.775810675320635
22.162835355742544
21.469348180112217
20.752814565284115
20.081272541927998
19.507753055200585
19.054054341004043
18.713342363340068
18.463913250052062
18.281766591342414
18.146851097583014
18.044389489285447
17.964075613251232
17.898848293409003
17.843861138136937
17.79574963261016
17.752139970039405
17.711325873158746
17.67205636467517
17.6333959618375
17.594632309130166
17.555215146646834
17.514716165860097
17.472802886070973
17.42922195792773
17.383788746125788


In [ ]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

In [ ]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [ ]:
comparison_data